# 3. Addresses for Servier
This file has fallen out of order. Run it after cleaning the dataset. The Servier data has no addresses, so we make a fuzzy search on all the others but with other criteria than the final fuzzy search. And we check it by hand

In [18]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import time

import sys
sys.path.insert(0, '../../data/lib/')

import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Read Data

In [34]:
#df = pd.read_csv('2. list_expanded.csv')
df = pd.read_csv('../../data/3. transformation/2. list_expanded.csv')
df_hcp = df[df.type == 'hcp'].copy()

#Reset index
df_hcp = df_hcp.reset_index(drop=True)

#Set Startindex to 1
df_hcp.index += 1 
len(df_hcp)

6125

## Calc rows

In [35]:
#Add Parent
df_hcp['parent'] = 0
df_hcp['parent'] = df_hcp['parent'].astype(int)
df_hcp['address_new'] = np.NaN

#Convert
df_hcp['name'] = df_hcp['name'].astype("str")
df_hcp['address_expand'] = df_hcp['address_expand'].astype("str")
df_hcp['location_expand'] = df_hcp['location_expand'].astype("str")

df_hcp['adress_new'] = df_hcp['address']

start_time = time.time()

for index, row in df_hcp[(df_hcp.source == 'servier') & df_hcp.address.isna()].iterrows():
    print(index)
    
    #Calculate ratio for each category
    df_hcp['r_name'] = df_hcp['name'].apply(lambda x: fuzz.token_set_ratio(x.lower(), row['name'].lower()))
    df_hcp['r_location'] = df_hcp['location_expand'].apply(lambda x: fuzz.token_set_ratio(x, row['location_expand']))
    #df_hcp['r_address'] = df_hcp['address_expand'].apply(lambda x: fuzz.token_set_ratio(x, row['address_expand']))
    
    df_hcp['r_total'] = df_hcp['r_name'] + df_hcp['r_location']# + df_hcp['r_address']
    df_hcp.loc[index, 'r_total'] = 0
    #Set the rules.
    #df_hcp.loc[(df_hcp.r_name >= 80) & (df_hcp.r_location >= 85) & (df_hcp.r_address >= 80), 'parent'] = index
    
    #Find the no1
    largest = df_hcp.nlargest(1, columns=['r_total'])
    df_hcp.loc[index, 'name_new'] = largest.iloc[0]['name']
    df_hcp.loc[index, 'address_new'] = largest.iloc[0]['address']
    df_hcp.loc[index, 'location_new'] = largest.iloc[0]['location']
    print(largest.iloc[0]['address'])
    #print(largest.iloc[0])
    
    
elapsed_time = time.time() - start_time
print('Finished in: ' + str(round(elapsed_time / 60, 2)) + ' minutes')


5288
Universitätsspital  Rämistrasse 100
5297
Alte Jonastrasse 24
5299
Bubenbergplatz 8 + 11
5300
Via Ospedale
5301
Via Stazione 29
5303
Studio Dr. Roggero Enrico Viale Portone 5
5304
Spital Limmattal Urdorferstrasse 100
5305
Tellstrasse
5306
Rue de l'hôpital 18
5307
Route de Chavannes 11A
5310
Via Cantonale 35d
5311
Via San Salvatore 7
5312
Clinica Luganese Moncucco SA Via Moncucco 10
5313
Clinica Luganese Moncucco SA Via Moncucco 10
5316
Avenue Grand-Champsec 80
5317
Kantonsspital  Medizin Onkologie, Pfaffenholzstras se 4
5318
FOSUMOS
5321
Mühlethalstrasse 27
5323
Rue Beausite 49
5325
St. Gallerstrasse 50
5326
Via Maggio 13
5327
USZ, Frauenklinikstrasse 26
5328
Route de Chavannes 9A / 11
5329
Avenue Cardinal-Mermillod 1
5332
Rigistrasse 1
5333
Chemin de l'Hôpital 9
5334
Avenue de la Fusion 27
5336
Centre Medical de  CMM, Promenade des Artisans 24
5338
Habich-Dietschy-Strasse 18
5339
Via Parco 3
5340
Via Tesserete 48
5343
Rue de la Maladière 45
5344
Optingenstrasse 8
5346
er Kantonssp

In [36]:
#df_hcp.to_csv('3. hcp_matches.csv', index=True)
servier = df_hcp[df_hcp.source=='servier']

write_to_excel(servier, 'servier.xlsx', open=True)
